### Flexibilizando la decision de trabajo

%%latex
\section*{Modelo de Equilibrio de Mercado con Ocio y Dotación de Tiempo}

Este modelo amplía el análisis económico al incluir la decisión del hogar respecto al ocio y considerar una dotación total de tiempo $T_{\text{bar}}$.

\subsection*{Problema del Hogar}
El hogar decide cómo dividir su dotación total de tiempo entre trabajo ($L$), ocio, y cómo distribuir su ingreso entre el consumo de dos bienes, maximizando su utilidad sujeta a una restricción presupuestaria.

\textbf{Función de Utilidad:}
La utilidad ahora también depende del ocio ($O$):
$$ U(x_1, x_2, O) = x_1^{\alpha_1} \cdot x_2^{\alpha_2} \cdot O^{1-\alpha_1-\alpha_2} $$

\textbf{Restricción Presupuestaria y de Tiempo:}
La restricción presupuestaria se ajusta para reflejar el ingreso proveniente del trabajo y el capital:
$$ p_1 x_1 + p_2 x_2 = wL + rK_{\text{bar}} $$
Además, la dotación total de tiempo se divide entre trabajo y ocio:
$$ L + O = T_{\text{bar}} $$

\textbf{Condiciones de Primer Orden (CPO):}
Las CPO para la maximización de la utilidad del hogar incorporan el ocio:
\begin{align*}
\frac{\partial U}{\partial x_1} &= \lambda p_1, \\
\frac{\partial U}{\partial x_2} &= \lambda p_2, \\
\frac{\partial U}{\partial O} &= \lambda w,
\end{align*}
donde $\lambda$ es el multiplicador de Lagrange de la restricción presupuestaria.

\subsection*{Problema de la Empresa}
La empresa busca maximizar sus beneficios eligiendo la cantidad de producción de cada bien y la cantidad de cada factor de producción a utilizar. Las funciones de producción y las condiciones de primer orden permanecen sin cambios respecto a la descripción anterior.

\subsection*{Condiciones de Equilibrio del Mercado y Ecuaciones Finales}
El equilibrio en los mercados de bienes y factores se alcanza cuando la oferta iguala la demanda en cada uno:

\textbf{Mercado de Bienes:}
\begin{align*}
Q_1 &= x_1, \\
Q_2 &= x_2.
\end{align*}

\textbf{Mercado de Factores y Ocio:}
\begin{align*}
L &= L_{\text{demandado}}, \\
O &= T_{\text{bar}} - L.
\end{align*}

Las ecuaciones finales que representan este sistema en el código de Julia forman un sistema de ecuaciones no lineales que incluyen las decisiones óptimas de producción y consumo, ajustadas para incorporar la decisión sobre el ocio.

Para resolver el sistema y encontrar los precios de equilibrio ($p_1$, $p_2$), el salario ($w$), el retorno al capital ($r$), y la asignación óptima de tiempo entre trabajo y ocio, utilizamos métodos numéricos como el paquete \texttt{NLsolve} en Julia. Este enfoque demuestra la capacidad de las herramientas computacionales para encontrar soluciones a modelos económicos complejos que incluyen múltiples decisiones y restricciones.


%%latex
\subsection*{Ecuaciones Finales del Mercado y Formación del Sistema}

Las ecuaciones finales del mercado, tal como se implementan en el código de Julia, forman un sistema de ecuaciones no lineales que representan el equilibrio en los mercados de bienes y factores. Este sistema se basa en las condiciones de primer orden y las condiciones de equilibrio del mercado.

\textbf{Ecuaciones Implementadas en Julia:}
\begin{align*}
\text{Ecuación 1: } & \frac{1}{p_1} - \left(\frac{b_1}{w}\right)^{b_1} \cdot \left(\frac{1-b_1}{r}\right)^{1-b_1} = 0, \\
\text{Ecuación 2: } & \frac{1}{p_2} - \left(\frac{b_2}{w}\right)^{b_2} \cdot \left(\frac{1-b_2}{r}\right)^{1-b_2} = 0, \\
\text{Ecuación 3: } & b_1 \cdot \alpha_1 \cdot \frac{Y_{\text{bar}}}{w} + b_2 \cdot \alpha_2 \cdot \frac{Y_{\text{bar}}}{w} + (1-\alpha_1-\alpha_2) \cdot \frac{Y_{\text{bar}}}{w} - T_{\text{bar}} = 0,
\end{align*}
donde $Y_{\text{bar}} = w \cdot T_{\text{bar}} + r \cdot K_{\text{bar}}$ es el ingreso total.

\textbf{Formación del Sistema de Ecuaciones:}
Este sistema de ecuaciones encapsula tanto las decisiones óptimas de producción de la empresa como las decisiones de consumo y oferta de trabajo del hogar. Al resolver este sistema, podemos encontrar los precios de equilibrio ($p_1$, $p_2$), el salario ($w$), y el retorno al capital ($r$) que equilibran todos los mercados simultáneamente.

Para resolver el sistema en Julia, utilizamos el paquete \texttt{NLsolve}, que busca un vector de valores $[p_2, w, r]$ que satisface todas las ecuaciones anteriores simultáneamente.

In [3]:
using NLsolve  # Importamos el paquete NLsolve para resolver sistemas no lineales

# Definimos los parámetros y dotaciones del modelo, incluyendo los nuevos
b1, b2 = 0.3, 0.6  # Parámetros para las funciones Cobb-Douglas
Kbar = 10.0    # Dotaciones totales de trabajo y capital
Tbar = 30.0                # Dotación total de trabajo decidida por el hogar
α1, α2 = 0.3, 0.4          # Nuevos parámetros para las preferencias del consumo

# Definimos la función 'mercados' con el modelo actualizado
function mercados(x)
    p = [1.0, x[1]]  # Vector de precios donde p[1] es normalizado a 1
    w = x[2]         # Salario
    r = x[3]         # Retorno del capital

    # Calculamos el ingreso total Ybar, considerando la decisión laboral del hogar
    Ybar = w * Tbar + r * Kbar

    # Ecuaciones del mercado incorporando la decisión laboral del hogar
    ecuaciones_mercado = zeros(3)
    ecuaciones_mercado[1] = 1.0 / p[1] - (b1 / w)^b1 * ((1.0 - b1) / r)^(1.0 - b1)
    ecuaciones_mercado[2] = 1.0 / p[2] - (b2 / w)^b2 * ((1.0 - b2) / r)^(1.0 - b2)
    ecuaciones_mercado[3] = b1 * α1 * Ybar / w + b2 * α2 * Ybar / w + (1.0 - α1 - α2) * Ybar / w - Tbar

    return ecuaciones_mercado
end

# Adivinanza inicial para el solucionador
x_inicial = [0.5, 0.5, 0.5]  # Ajuste según sea necesario

# Resolvemos las ecuaciones del mercado
resultado = nlsolve(mercados, x_inicial)

# Verificamos la convergencia y mostramos resultados
if converged(resultado)
    println("Solución encontrada: ", resultado.zero)
else
    error("Solución no encontrada: el solucionador no convergió.")
end

Solución encontrada: [0.8978329274627539, 0.3651886773717779, 0.6434276697711006]


### Sector Publico e Incidencia de Sistema Impositivos

%%latex
\section*{Modelo de Equilibrio de Mercado Extendido con Gobierno y Impuestos}

Este modelo incorpora el gobierno, impuestos sobre el consumo y la renta, y precios diferenciados para productores y consumidores.

\subsection*{Problema del Hogar}
El hogar decide cómo asignar su tiempo entre trabajo y ocio, y cómo distribuir su ingreso disponible entre el consumo de bienes.

\textbf{Función de Utilidad:}
$$ U(x_1, x_2, \ell) = x_1^{\alpha_1} \cdot x_2^{\alpha_2} \cdot \ell^{1-\alpha_1-\alpha_2} $$

\textbf{Restricción Presupuestaria:}
$$ p_1 x_1 + p_2 x_2 + w^n \ell = w^n T_{\text{bar}} + r^n K_{\text{bar}} $$
donde $w^n = w(1-\tau_w)$ y $r^n = r(1-\tau_r)$ son los salarios y retornos de capital netos de impuestos, respectivamente.

\subsection*{Problema de la Empresa}
La empresa elige la producción para maximizar beneficios, considerando los precios de productores y los costos de los factores de producción.

\textbf{Funciones de Producción:}
\begin{align*}
Y_1 &= L^{b_1} K^{1-b_1}, \\
Y_2 &= L^{b_2} K^{1-b_2}.
\end{align*}

\subsection*{Gobierno y Restricciones}
El gobierno consume una cantidad exógena $G$ del bien 1 y recauda impuestos sobre el consumo y la renta.

\textbf{Ingresos y Gastos del Gobierno:}
\begin{align*}
\text{Ingresos: } & \tau_{c} p x + \tau_{w} w L + \tau_{r} r K_{\text{bar}}, \\
\text{Gastos: } & q_1 G.
\end{align*}

\subsection*{Condiciones de Equilibrio del Mercado}
Las condiciones de equilibrio se ajustan para incluir el consumo y la intervención del gobierno.

\textbf{Mercado de Bienes:}
\begin{align*}
X_1 + G &= Y_1, \\
X_2 &= Y_2.
\end{align*}

\textbf{Mercado de Factores:}
\begin{align*}
L &= L_{\text{demandado}}, \\
K &= K_{\text{bar}}.
\end{align*}

\subsection*{Ecuaciones Finales del Mercado}
Las ecuaciones finales del mercado en el código de Julia reflejan las decisiones de los hogares, las empresas y el gobierno, así como las condiciones de equilibrio en los mercados de bienes y factores:

\begin{align*}
\text{Ecuación 1 (Bien 1): } & \alpha_1 \frac{Y_{\text{barn}}}{p_1} + G - \left(\frac{b_1}{w^n}\right)^{b_1} \left(\frac{1-b_1}{r^n}\right)^{1-b_1} q_1 \left(\alpha_1 \frac{Y_{\text{barn}}}{p_1} + G\right) = 0, \\
\text{Ecuación 2 (Bien 2): } & \frac{1}{p_2} - \left(\frac{b_2}{w^n}\right)^{b_2} \left(\frac{1-b_2}{r^n}\right)^{1-b_2} \frac{q_2}{p_2} = 0, \\
\text{Ecuación 3 (Mercado Laboral): } & \frac{b_1}{w^n} q_1 \left(\alpha_1 \frac{Y_{\text{barn}}}{p_1} + G\right) + \frac{b_2}{w^n} q_2 \alpha_2 \frac{Y_{\text{barn}}}{p_2} + (1-\alpha_1-\alpha_2) \frac{Y_{\text{barn}}}{w^n} - T_{\text{bar}} = 0, \\
\text{Ecuación 4 (Presupuesto del Gobierno): } & q_1 G - \frac{\tau_{\text{c1}}}{1+\tau_{\text{c1}}} \alpha_1 Y_{\text{barn}} - \frac{\tau_{\text{c2}}}{1+\tau_{\text{c2}}} \alpha_2 Y_{\text{barn}} - \tau_{\text{w}} w (T_{\text{bar}} - \frac{1-\alpha_1-\alpha_2}{w^n} Y_{\text{barn}}) - \tau_{\text{r}} r K_{\text{bar}} = 0.
\end{align*}

En estas ecuaciones, $Y_{\text{barn}}$ representa el ingreso neto total del hogar, y $p_1$, $p_2$ son los precios al consumidor de los bienes. Estas ecuaciones forman un sistema que, al ser resuelto, proporciona los precios de equilibrio en los mercados, así como los niveles óptimos de producción, consumo y trabajo, teniendo en cuenta la intervención del gobierno y la política fiscal.

Este modelo extendido proporciona una visión completa de la interacción entre hogares, empresas y gobierno en un contexto de mercados con impuestos y gastos gubernamentales. La resolución de este modelo en Julia a través de métodos numéricos permite a los estudiantes explorar el impacto de políticas fiscales y decisiones gubernamentales en un entorno económico complejo.

In [6]:
using NLsolve  # Importamos el paquete NLsolve

# Definimos los parámetros, tasas de impuestos, consumo del gobierno, etc.
α1, α2 = 0.3, 0.4          # Parámetros para las preferencias de consumo
b1, b2 = 0.3, 0.6          # Parámetros para funciones Cobb-Douglas
Kbar = 10.0                # Dotación de capital
Tbar = 30.0                # Dotación total de tiempo (horas)
G = 5.0                    # Consumo gubernamental del bien 1
τc1 = 0.1  # Tasa de impuesto al consumo para el bien 1
τc2 = 0.15 # Tasa de impuesto al consumo para el bien 2
tauc = [τc1, τc2]          # Tasas de impuestos al consumo para los bienes 1 y 2
tauw = 0.2                 # Tasa de impuesto sobre salarios

# Definimos la función 'mercados' con el modelo actualizado
function mercados(x)
    # Extraemos los valores de las variables
    q = [1.0, x[1]]  # Precios de productores
    w, r, taur = x[2], x[3], x[4]

    # Verificar que 'w' y 'r' sean positivos
    if w <= 0 || r <= 0
        return fill(1e10, 4)  # Retornar un vector con un número grande para indicar error
    end

    # Calcular precios al consumidor y ingreso neto
    p = q .* (1.0 .+ tauc)  # Precios al consumidor
    wn = w * (1.0 - tauw)   # Salario neto
    rn = r * (1.0 - taur)   # Retorno sobre capital neto
    Ybarn = wn * Tbar + rn * Kbar  # Ingreso total neto

    # Ecuaciones de mercado con impuestos y consumo gubernamental
    ecuaciones_mercado = zeros(4)
    ecuaciones_mercado[1] = α1 * Ybarn / p[1] + G - (b1 / w)^b1 * ((1.0 - b1) / r)^(1.0 - b1) * q[1] * (α1 * Ybarn / p[1] + G)
    ecuaciones_mercado[2] = 1.0 / p[2] - (b2 / w)^b2 * ((1.0 - b2) / r)^(1.0 - b2) * q[2] / p[2]
    ecuaciones_mercado[3] = b1 / w * q[1] * (α1 * Ybarn / p[1] + G) + b2 / w * q[2] * α2 * Ybarn / p[2] + (1.0 - α1 - α2) * Ybarn / wn - Tbar
    ecuaciones_mercado[4] = q[1] * G - tauc[1] / (1.0 + tauc[1]) * α1 * Ybarn - tauc[2] / (1.0 + tauc[2]) * α2 * Ybarn - tauw * w * (Tbar - (1.0 - α1 - α2) / wn * Ybarn) - taur * r * Kbar

    return ecuaciones_mercado
end

# Adivinanza inicial
x_inicial = [1.0, 1.0, 1.0, 0.1]  # Ajustar según sea necesario

# Resolver las ecuaciones de mercado
resultado = nlsolve(mercados, x_inicial)

# Verificar la convergencia y mostrar resultados
if converged(resultado)
    println("Solución encontrada: ", resultado.zero)
else
    error("Solución no encontrada: el solucionador no convergió.")
end

Solución encontrada: [0.8148770041873808, 0.2912566712351418, 0.7089297469600773, 0.45196314417406724]
